In [6]:
!pip install semantic-kernel==0.9.1b1

In [10]:
import os
import semantic_kernel as sk
import json

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planners.action_planner import ActionPlanner

plugins_directory = "plugins"
plugin_names = [plugin for plugin in os.listdir(plugins_directory) if os.path.isdir(os.path.join(plugins_directory, plugin))]
#plugin_names = ["PhilPlugin"]

print("Plugins found: ", plugin_names)

Plugins found:  ['EditingPlugin']


In [11]:
kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
service_id = "default"

kernel.add_service(
    AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
)

for plugin_name in plugin_names:
    print("Loading plugin: ", plugin_name)
    plugin = kernel.import_plugin_from_prompt_directory(plugins_directory, plugin_name)
    plugin.description = "This is the editing plugin."
   
# convert plugin to json string
plugin_json = json.dumps(plugin, default=lambda x: x.__dict__)
print(plugin_json)

   


Loading plugin:  EditingPlugin
{"name": "EditingPlugin", "description": "This is the editing plugin.", "functions": {"ChangeTense": {"metadata": {"name": "ChangeTense", "plugin_name": "EditingPlugin", "description": "Takes a text and changes it to a specified verbal tense.i.e. Present, Past, Future, etc.", "parameters": [{"name": "input", "description": "Text to be processed.", "default_value": "", "type_": "", "is_required": true, "type_object": null}, {"name": "tense", "description": "Tense we want to set the input to.", "default_value": "present", "type_": "", "is_required": true, "type_object": null}], "is_prompt": true, "is_asynchronous": true, "return_parameter": {"name": "return", "description": "The completion result", "default_value": null, "type_": "FunctionResult", "is_required": true, "type_object": null}}, "prompt_template": {"prompt_template_config": {"name": "ChangeTense", "description": "Takes a text and changes it to a specified verbal tense.i.e. Present, Past, Future,

In [3]:
ask = """Change the following sentence to past tense:
I am going to the mall tomorrow to buy some clothes."""

#ask = """Change the following paragraph to past tense:
#Southwest, which flies only Boeing planes, is particularly exposed to the manufacturer’s struggles. Southwest said its plans to cut flying capacity will primarily happen in the second half of the year. The carrier said it is still in the process of reworking plans but will reduce pilot hiring by more than 50% and reduce flight-attendant hiring by more than 60% this year, looking to reduce costs."""

In [12]:
planner = ActionPlanner(kernel, service_id=service_id)

plan = await planner.create_plan(goal=ask)

print(plan.function)

Overwriting plugin "ActionPlanner_Excluded" in collection


metadata=KernelFunctionMetadata(name='ChangeTense', plugin_name='EditingPlugin', description='Takes a text and changes it to a specified verbal tense.i.e. Present, Past, Future, etc.', parameters=[KernelParameterMetadata(name='input', description='Text to be processed.', default_value='', type_='', is_required=True, type_object=None), KernelParameterMetadata(name='tense', description='Tense we want to set the input to.', default_value='present', type_='', is_required=True, type_object=None)], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None)) prompt_template=KernelPromptTemplate(prompt_template_config=PromptTemplateConfig(name='ChangeTense', description='Takes a text and changes it to a specified verbal tense.i.e. Present, Past, Future, etc.', template='USE THE INPUT TEXT AND REWRITE IT CHANGING ONLY THE VERBAL TENSE, IF PROVID

In [13]:
results = await plan.invoke(kernel)

In [14]:
print(results)

I was going to the mall the next day to buy some clothes.
